Stage 1: Data crawler. Massively parallel functions crawling data and storing it in Cloud Object Storage.  Obtain information from web pages or tweets and create a dataset of text data. Use FaaS backend in lithops to launch crawling process over serverless functions.

In [101]:
import lithops


In [102]:
from custom_snscrape.twitter import TwitterSearchScraper

# TODO: Improve function names...
# TODO: Read this https://github.com/lithops-cloud/lithops/blob/master/docs/api_storage.md
# TODO: put_object vs put_cloudobject?

def scrapSearch(str_to_search, storage):
    for tweet in TwitterSearchScraper(str_to_search).get_items():
        print(str_to_search)
        # key_prefix is only the included hashtag, without the excluded ones
        key_prefix = str_to_search.split()[0]

        bucket = storage.storage_config['bucket']
        storage.put_object(bucket=bucket,
                              key=key_prefix+'/'+str(tweet.id),
                              body=tweet.to_json())
        
        break # testing: only 1 result

    return "OK"

In [112]:
import ipywidgets as widgets
items = [widgets.Text(placeholder='Hastag #{}'.format(i)) for i in range(5)]
widgets.HBox(items)

In [113]:
with lithops.FunctionExecutor() as fexec:
    # Each query will include the current hashtag,
    # and exclude the hashtags to the right of itself in the array.
    # This way, we can parallelise the searches, while also ensuring that
    # if a tweet contains multiple hashtags, one of the queries includes said tweet.
    # 
    # In this case, the queries will be:
    # #covid -#sars -#coronavirus, #sars -#coronavirus, #coronavirus
    #hashtags = ['#covid', '#sars', '#coronavirus'] # TODO: Use a jupyter widget (textbox perhaps?) to insert here the hashtags.

    hashtags = []
    for i in items:
        if i.value != '':
            hashtags.append(i.value)

    queries = []

    for i in range(len(hashtags)):
        queries.append(' -'.join(hashtags))
        hashtags.pop(0)

    all_invocations = fexec.map(scrapSearch, queries)
    print(fexec.get_result())

2021-05-24 18:25:13,041 [INFO] lithops.config -- Lithops v2.3.3
2021-05-24 18:25:13,044 [INFO] lithops.storage.backends.localhost.localhost -- Localhost Storage client created
2021-05-24 18:25:13,045 [INFO] lithops.localhost.localhost -- Localhost compute client created
2021-05-24 18:25:13,046 [INFO] lithops.executors -- Localhost Executor created with ID: bc95cb-28
2021-05-24 18:25:13,049 [INFO] lithops.invokers -- ExecutorID bc95cb-28 | JobID M000 - Selected Runtime: python.exe 
2021-05-24 18:25:13,078 [INFO] lithops.job.job -- ExecutorID bc95cb-28 | JobID M000 - Uploading function and data - Total: 45.0KiB
2021-05-24 18:25:13,085 [INFO] lithops.invokers -- ExecutorID bc95cb-28 | JobID M000 - Starting function invocation: scrapSearch() - Total: 5 activations
2021-05-24 18:25:13,093 [INFO] lithops.invokers -- ExecutorID bc95cb-28 | JobID M000 - View execution logs at C:\Users\jordi\AppData\Local\Temp\lithops\logs\bc95cb-28-M000.log
2021-05-24 18:25:13,100 [INFO] lithops.wait -- Execut

In [105]:
Tweets: 
1) Pedro Sanchez
 https://twitter.com/sanchezcastejon/status/1388184494235127814
 https://twitter.com/sanchezcastejon/status/1376241393396543492
 https://twitter.com/sanchezcastejon/status/1371013139987365888

2)  Boris Johnson
https://twitter.com/BorisJohnson/status/1391413912889921542
https://twitter.com/BorisJohnson/status/1363552116133023744
https://twitter.com/BorisJohnson/status/1354147835369959435

3) Joe Biden
https://twitter.com/JoeBiden/status/1386426190861197313
https://twitter.com/POTUS/status/1373037818277167112
https://twitter.com/POTUS/status/1384136220284915712
https://twitter.com/POTUS/status/1368993395172139014

4) Elon Musk
https://twitter.com/elonmusk/status/1256240766600409088




SyntaxError: invalid syntax (<ipython-input-105-0fbccafb20b4>, line 1)